In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

from tqdm import tqdm
from openai import OpenAI
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
class CFG:
  # Modelo
  PATH = "/content/drive/MyDrive/Melina - Aex/aex_estruturados.parquet"
  API_KEY = "."

  # Embeddings
  MODEL = "text-embedding-3-large"

  THRESHOLD = .9 # Threshold para a similaridade

In [ ]:
class Processor:
  def __init__(self):
    self.key = CFG.API_KEY
    self.client = OpenAI(api_key=self.key)
    self.data = pd.read_parquet(CFG.PATH)

    self.run()

  @property
  def get_data(self):
    return self.data.copy()

  def run(self):
    self.build_corpus()
    self.get_embeddings()

  def build_corpus(self):
    self.data["Corpus"] = (
        self.data["Título da Atividade"].fillna(" ") + " " +
        self.data["Descrição da Atividade"].fillna(" ")
    ).str.strip()

    self.data["Corpus"] = self.data["Corpus"].str.replace("\n", " ")

  def get_embeddings(self):
     self.data["Embedding"] = self.data["Corpus"].progress_apply(
        lambda x: self.client.embeddings.create(
            model=CFG.MODEL,
            input=x
        ).data[0].embedding
    )

In [ ]:
def find_similar(df, similarity_matrix, idx, threshold):
  similar_indices = np.where(similarity_matrix[idx] >= threshold)[0]
  similar_indices = similar_indices[similar_indices != idx]
  return df.loc[similar_indices, "AEX ID"].tolist()

def cos_sim_by_nusp(df):
    dataframes = []

    for nusp in tqdm(df["Docente Responsável - NUSP"].unique()):
        temp = df[df["Docente Responsável - NUSP"] == nusp].copy()
        temp.reset_index(inplace=True, drop=True)

        if len(temp) == 1:
            temp["Similares"] = [[]]
        else:
            matrix = cosine_similarity(np.vstack(temp.Embedding.values))
            temp["Similares"] = temp.index.map(lambda idx: find_similar(temp, matrix, idx, CFG.THRESHOLD))

        dataframes.append(temp[["Título da Atividade", "AEX ID", "Docente Responsável - Nome", "Similares"]])

    return pd.concat(dataframes)

def flatten_by_similarity(df, similars):
  '''
    Recebe um dataframe contendo AEX similares (similars)
    e o conjunto total de dados AEX (df)
    Retorna uma versão compactada de df com base na similaridade
  '''
  df = df.copy()

  added = []
  to_remove = []

  for key, row in similars.iterrows():
      IDs = row["Similares"]
      root = row["AEX ID"]

      if root not in added:
          added.append(root)
          for i, ID in enumerate(IDs):
              added.append(ID)
              to_remove.append(ID)
              similar_row = df.loc[df["AEX ID"] == ID, ["AEX ID", "Título da Atividade", "Descrição da Atividade"]]

              for col in similar_row.columns:
                  df.loc[df["AEX ID"] == root, f"Similar_{i+1}_{col}"] = similar_row.iloc[0][col] if not similar_row.empty else None

  df = df[~df["AEX ID"].isin(to_remove)]

  return df

def run(df):
  df = df.copy()
  df = flatten_by_similarity(df, cos_sim_by_nusp(df))

  return df

In [ ]:
handler = Processor()

100%|██████████| 1179/1179 [10:35<00:00,  1.85it/s]


In [ ]:
data = handler.get_data

In [ ]:
agregados = run(data)
repetidos = agregados[~agregados["Similar_1_AEX ID"].isna()].copy()

100%|██████████| 723/723 [00:01<00:00, 533.76it/s]


In [ ]:
repetidos = repetidos[[
 'Docente Responsável - NUSP',
 'Docente Responsável - Nome',
 'Unidade Nome',
 'Unidade Sigla',

 'Título da Atividade',
 'Similar_1_Título da Atividade',
 'Similar_2_Título da Atividade',
 'Similar_3_Título da Atividade',
 'Similar_4_Título da Atividade',
 'Similar_5_Título da Atividade',

 'Descrição da Atividade',
 'Similar_1_Descrição da Atividade',
 'Similar_2_Descrição da Atividade',
 'Similar_3_Descrição da Atividade',
 'Similar_4_Descrição da Atividade',
 'Similar_5_Descrição da Atividade',

  'AEX ID',
 'Similar_1_AEX ID',
 'Similar_2_AEX ID',
 'Similar_3_AEX ID',
 'Similar_4_AEX ID',
 'Similar_5_AEX ID',
]].copy()

repetidos = repetidos.rename(columns={
    'Docente Responsável - NUSP': 'Docente Responsável - NUSP (ROOT)',
    'Docente Responsável - Nome': 'Docente Responsável - Nome (ROOT)',
    'Unidade Nome': 'Unidade Nome (ROOT)',
    'Unidade Sigla': 'Unidade Sigla (ROOT)',
    'AEX ID': 'AEX ID (ROOT)',
    'Título da Atividade': 'Título da Atividade (ROOT)',
    'Descrição da Atividade': 'Descrição da Atividade (ROOT)'
})

repetidos.reset_index(inplace=True, drop=True)

In [ ]:
repetidos.to_csv("repetidos.csv", index=False)

In [ ]:
data.to_parquet("/content/drive/MyDrive/Melina - Aex/embeddings_aex.parquet")